# 🎓 Soluciones: Ejercicios de Econometría
### Basados en Wooldridge: Introducción a la Econometría (4ta Edición)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IreneZMo/ejercicios_econometria/blob/main/soluciones_ejercicios_econometria.ipynb)

---

**Contenido:**
1. Regresión Simple: Sueño y Horas de Trabajo
2. Regresión Múltiple: Peso al Nacer y Tabaquismo  
3. Inferencia Estadística: Riqueza Financiera, Ingreso y Edad

---

## ⚙️ Configuración Inicial

Ejecuta esta celda primero para instalar las librerías necesarias y descargar los datos.

In [ ]:
# ============================================================================
# INSTALACIÓN DE LIBRERÍAS (solo necesario en Google Colab)
# ============================================================================

# Instalamos el paquete wooldridge que contiene todos los datasets del libro
!pip install wooldridge -q

# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import wooldridge as woo

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', lambda x: '%.4f' % x)
%matplotlib inline

print("✅ Librerías cargadas correctamente!")

In [ ]:
# ============================================================================
# CARGA DE DATASETS
# ============================================================================

# El paquete 'wooldridge' contiene todos los datasets del libro
# Documentación: https://pypi.org/project/wooldridge/

# Dataset para Ejercicio 1: Sueño y trabajo
sleep75 = woo.data('sleep75')
print(f"📊 sleep75: {sleep75.shape[0]} observaciones, {sleep75.shape[1]} variables")

# Dataset para Ejercicio 2: Peso al nacer
bwght = woo.data('bwght')
print(f"📊 bwght: {bwght.shape[0]} observaciones, {bwght.shape[1]} variables")

# Dataset para Ejercicio 3: Planes 401k
ksubs = woo.data('401ksubs')
print(f"📊 401ksubs: {ksubs.shape[0]} observaciones, {ksubs.shape[1]} variables")

print("\n✅ Datos cargados correctamente!")

---
# 📘 EJERCICIO 1: Regresión Simple
## Sueño y Horas de Trabajo (Capítulo 2)

**Contexto:** Biddle y Hamermesh (1990) estudiaron la relación entre las horas de sueño y las horas de trabajo.

**Modelo:** `sleep = β₀ + β₁·totwrk + u`

**Variables:**
- `sleep`: Minutos de sueño por semana
- `totwrk`: Minutos de trabajo por semana

In [ ]:
# ============================================================================
# EXPLORACIÓN DE DATOS
# ============================================================================

print("=" * 60)
print("EJERCICIO 1: REGRESIÓN SIMPLE - Sueño y Horas de Trabajo")
print("=" * 60)

# Seleccionamos las variables relevantes
sleep = sleep75[['sleep', 'totwrk', 'age', 'educ', 'male']].copy()

print(f"\nDimensiones: {sleep.shape[0]} observaciones, {sleep.shape[1]} variables")
print("\nPrimeras observaciones:")
display(sleep.head())

print("\nEstadísticas descriptivas:")
display(sleep.describe())

### Pregunta a) Estimación del modelo de regresión simple

En regresión simple, buscamos la línea que minimiza la **suma de los residuales al cuadrado** (MCO = Mínimos Cuadrados Ordinarios).

⚠️ **IMPORTANTE:** `statsmodels` NO incluye constante por defecto. Hay que usar `sm.add_constant()`

In [ ]:
# ============================================================================
# PREGUNTA a) ESTIMACIÓN DEL MODELO
# ============================================================================

# Variable dependiente (Y): sleep
y = sleep['sleep']

# Variable independiente (X): totwrk
X = sleep['totwrk']

# ⚠️ AGREGAMOS LA CONSTANTE - MUY IMPORTANTE
# Sin esto, estaríamos forzando la línea a pasar por el origen
X = sm.add_constant(X)

print("Matriz de diseño X (primeras 5 filas):")
display(X.head())

# Estimamos el modelo por MCO (OLS = Ordinary Least Squares)
modelo1 = sm.OLS(y, X).fit()

# Mostramos los resultados completos
print("\n" + "=" * 60)
print("RESULTADOS DE LA REGRESIÓN")
print("=" * 60)
print(modelo1.summary())

In [ ]:
# Extraemos los coeficientes para interpretación
beta_0 = modelo1.params['const']      # Intercepto
beta_1 = modelo1.params['totwrk']     # Pendiente
r_squared = modelo1.rsquared          # R-cuadrada
n = modelo1.nobs                      # Número de observaciones

print("\n" + "=" * 60)
print("ECUACIÓN ESTIMADA")
print("=" * 60)
print(f"\nsleep = {beta_0:.2f} + ({beta_1:.4f}) × totwrk")
print(f"\nn = {int(n)}, R² = {r_squared:.3f}")

### Pregunta b) Interpretación del intercepto

In [ ]:
# ============================================================================
# PREGUNTA b) INTERPRETACIÓN DEL INTERCEPTO
# ============================================================================

print(f"""
📌 INTERPRETACIÓN DEL INTERCEPTO
{"="*50}

El intercepto β₀ = {beta_0:.2f} representa los minutos de sueño predichos 
por semana cuando totwrk = 0 (es decir, cuando la persona NO trabaja).

Conversión a horas de sueño por noche:
• {beta_0:.2f} minutos por semana
• ÷ 7 días = {beta_0/7:.2f} minutos por noche
• ÷ 60 = {beta_0/7/60:.2f} horas por noche

✅ Esto es RAZONABLE: una persona que no trabaja dormiría aproximadamente 
   {beta_0/7/60:.1f} horas por noche, dentro del rango normal de 7-9 horas.

💡 NOTA: El intercepto tiene sentido práctico cuando X=0 está dentro del 
   rango de los datos o cerca de él.
""")

### Pregunta c) Efecto de 2 horas adicionales de trabajo

In [ ]:
# ============================================================================
# PREGUNTA c) EFECTO DE 2 HORAS ADICIONALES DE TRABAJO
# ============================================================================

# 2 horas = 120 minutos
delta_totwrk = 120  # minutos adicionales de trabajo
delta_sleep = beta_1 * delta_totwrk

print(f"""
📌 EFECTO MARGINAL
{"="*50}

El coeficiente β₁ = {beta_1:.4f} indica que por cada minuto adicional 
de trabajo por semana, el sueño disminuye en {abs(beta_1):.4f} minutos.

Cálculo para 2 horas (120 minutos) adicionales de trabajo:

    Δsleep = β₁ × Δtotwrk 
           = {beta_1:.4f} × 120 
           = {delta_sleep:.2f} minutos por semana

Esto equivale a:
• {abs(delta_sleep)/7:.2f} minutos menos de sueño POR NOCHE
• Aproximadamente {abs(delta_sleep)/7/60*60:.1f} minutos menos por noche

❓ ¿Es este efecto GRANDE?

El efecto es relativamente PEQUEÑO. Por cada hora adicional de trabajo 
semanal, solo se pierden {abs(beta_1)*60:.1f} minutos de sueño por semana.

💡 Esto sugiere que las personas ajustan OTRAS actividades (ocio, TV, etc.)
   más que su tiempo de sueño cuando trabajan más.
""")

### Pregunta d) Poder explicativo del modelo (R²)

In [ ]:
# ============================================================================
# PREGUNTA d) PODER EXPLICATIVO (R²)
# ============================================================================

print(f"""
📌 COEFICIENTE DE DETERMINACIÓN (R²)
{"="*50}

R² = {r_squared:.3f} = {r_squared*100:.1f}%

Interpretación:
El {r_squared*100:.1f}% de la variación en los minutos de sueño es explicada 
por las horas de trabajo.

❓ ¿Es esto MUCHO?

NO, es un R² bajo. El {(1-r_squared)*100:.1f}% de la variación en el sueño 
se debe a OTROS FACTORES no incluidos en el modelo:

• Salud física y mental
• Estrés
• Hábitos personales
• Edad y situación familiar
• Calidad del sueño vs cantidad
• etc.

⚠️ IMPORTANTE: Un R² bajo NO significa que el modelo sea inútil.
   El coeficiente de totwrk sigue siendo una estimación VÁLIDA del 
   efecto parcial del trabajo sobre el sueño.
""")

In [ ]:
# ============================================================================
# VISUALIZACIÓN - EJERCICIO 1
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Dispersión con línea de regresión
ax1 = axes[0]
ax1.scatter(sleep['totwrk'], sleep['sleep'], alpha=0.5, s=20, label='Datos observados')
x_line = np.linspace(sleep['totwrk'].min(), sleep['totwrk'].max(), 100)
y_line = beta_0 + beta_1 * x_line
ax1.plot(x_line, y_line, color='red', linewidth=2, 
         label=f'Línea de regresión\nŷ = {beta_0:.0f} + ({beta_1:.3f})×totwrk')
ax1.set_xlabel('Minutos de trabajo por semana (totwrk)', fontsize=11)
ax1.set_ylabel('Minutos de sueño por semana (sleep)', fontsize=11)
ax1.set_title('Ejercicio 1: Regresión Simple - Sueño vs Trabajo', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Residuales
ax2 = axes[1]
residuales = modelo1.resid
ax2.scatter(modelo1.fittedvalues, residuales, alpha=0.5, s=20)
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Valores ajustados (ŷ)', fontsize=11)
ax2.set_ylabel('Residuales (e = y - ŷ)', fontsize=11)
ax2.set_title('Análisis de Residuales', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 📘 EJERCICIO 2: Regresión Múltiple
## Peso al Nacer y Tabaquismo (Capítulo 3)

**Contexto:** Estudio sobre los efectos del tabaquismo durante el embarazo sobre el peso del bebé al nacer.

**Modelos:**
- Simple: `bwght = β₀ + β₁·cigs + u`
- Múltiple: `bwght = β₀ + β₁·cigs + β₂·faminc + u`

**Variables:**
- `bwght`: Peso al nacer (onzas)
- `cigs`: Cigarrillos por día durante el embarazo
- `faminc`: Ingreso familiar (miles de $)

In [ ]:
# ============================================================================
# EXPLORACIÓN DE DATOS
# ============================================================================

print("=" * 60)
print("EJERCICIO 2: REGRESIÓN MÚLTIPLE - Peso al Nacer y Tabaquismo")
print("=" * 60)

print(f"\nDimensiones: {bwght.shape[0]} observaciones")
print("\nPrimeras observaciones:")
display(bwght[['bwght', 'cigs', 'faminc', 'male', 'motheduc']].head())

print("\nEstadísticas descriptivas:")
display(bwght[['bwght', 'cigs', 'faminc']].describe())

# Estadísticas sobre fumadoras
fumadoras = (bwght['cigs'] > 0).sum()
print(f"\n📊 Madres que fumaron durante el embarazo: {fumadoras} ({fumadoras/len(bwght)*100:.1f}%)")
print(f"📊 Promedio de cigarrillos (solo fumadoras): {bwght[bwght['cigs']>0]['cigs'].mean():.1f}")

### Pregunta a) Signo esperado del coeficiente de faminc

In [ ]:
# ============================================================================
# PREGUNTA a) SIGNO ESPERADO DE β₂ (faminc)
# ============================================================================

print("""
📌 SIGNO ESPERADO DEL COEFICIENTE DE faminc
══════════════════════════════════════════════════════════════

Esperamos que β₂ (coeficiente de faminc) sea POSITIVO ➕

Razonamiento económico:

    Mayor ingreso familiar
           ↓
    ┌──────┴──────┐
    ↓             ↓
  Mejor        Mejor
  nutrición    cuidado prenatal
    ↓             ↓
    └──────┬──────┘
           ↓
    Mayor peso al nacer ✅

Además:
• Mayor ingreso → menos estrés financiero
• Mayor ingreso → posibilidad de reducir horas de trabajo
""")

### Pregunta b) Correlación entre cigs y faminc

In [ ]:
# ============================================================================
# PREGUNTA b) CORRELACIÓN ENTRE cigs Y faminc
# ============================================================================

correlacion = bwght['cigs'].corr(bwght['faminc'])

print(f"""
📌 CORRELACIÓN ENTRE VARIABLES INDEPENDIENTES
══════════════════════════════════════════════════════════════

Correlación muestral entre cigs y faminc: {correlacion:.4f}

La correlación es NEGATIVA ➖, lo cual tiene sentido:

• Familias con menores ingresos → mayores tasas de tabaquismo
• La educación (correlacionada con ingreso) → reduce probabilidad de fumar
• Mejor acceso a programas de cesación → con más recursos

⚠️ IMPLICACIÓN PARA EL SESGO POR VARIABLE OMITIDA:

Si NO controlamos por faminc:
• Parte del efecto negativo de cigs podría deberse a que 
  las fumadoras TAMBIÉN tienden a tener menores ingresos
• El modelo simple podría SOBREESTIMAR el efecto del tabaquismo
""")

# Visualización
fig, ax = plt.subplots(figsize=(8, 5))
fumadoras_df = bwght[bwght['cigs'] > 0]
ax.scatter(fumadoras_df['faminc'], fumadoras_df['cigs'], alpha=0.5)
ax.set_xlabel('Ingreso familiar (miles de dólares)')
ax.set_ylabel('Cigarrillos por día')
ax.set_title(f'Correlación entre Tabaquismo e Ingreso\n(solo fumadoras, r = {correlacion:.3f})')
plt.show()

### Pregunta c) Estimación de modelos simple y múltiple

In [ ]:
# ============================================================================
# PREGUNTA c) ESTIMACIÓN DE MODELOS
# ============================================================================

print("\n" + "=" * 60)
print("MODELO SIMPLE: bwght = β₀ + β₁·cigs + u")
print("=" * 60)

# MODELO SIMPLE
y = bwght['bwght']
X_simple = sm.add_constant(bwght['cigs'])
modelo_simple = sm.OLS(y, X_simple).fit()

print(modelo_simple.summary())

beta0_simple = modelo_simple.params['const']
beta1_simple = modelo_simple.params['cigs']
r2_simple = modelo_simple.rsquared

print(f"\n📊 Ecuación: bwght = {beta0_simple:.2f} + ({beta1_simple:.3f}) × cigs")
print(f"📊 n = {int(modelo_simple.nobs)}, R² = {r2_simple:.3f}")

In [ ]:
print("\n" + "=" * 60)
print("MODELO MÚLTIPLE: bwght = β₀ + β₁·cigs + β₂·faminc + u")
print("=" * 60)

# MODELO MÚLTIPLE
X_multiple = sm.add_constant(bwght[['cigs', 'faminc']])
modelo_multiple = sm.OLS(y, X_multiple).fit()

print(modelo_multiple.summary())

beta0_mult = modelo_multiple.params['const']
beta1_mult = modelo_multiple.params['cigs']
beta2_mult = modelo_multiple.params['faminc']
r2_mult = modelo_multiple.rsquared

print(f"\n📊 Ecuación: bwght = {beta0_mult:.2f} + ({beta1_mult:.3f}) × cigs + ({beta2_mult:.3f}) × faminc")
print(f"📊 n = {int(modelo_multiple.nobs)}, R² = {r2_mult:.3f}")

### Pregunta d) Análisis del sesgo por variable omitida

In [ ]:
# ============================================================================
# PREGUNTA d) SESGO POR VARIABLE OMITIDA
# ============================================================================

cambio_coef = beta1_mult - beta1_simple
cambio_pct = (cambio_coef / beta1_simple) * 100

print(f"""
📌 ANÁLISIS DEL SESGO POR VARIABLE OMITIDA
══════════════════════════════════════════════════════════════

COMPARACIÓN DE COEFICIENTES DE cigs:

                      Modelo Simple    Modelo Múltiple    Cambio
                      ─────────────    ───────────────    ──────
Coef. de cigs:        {beta1_simple:>10.3f}       {beta1_mult:>10.3f}      {cambio_coef:>+.3f} ({cambio_pct:+.1f}%)
R²:                   {r2_simple:>10.3f}       {r2_mult:>10.3f}      {r2_mult-r2_simple:>+.3f}

══════════════════════════════════════════════════════════════

📐 FÓRMULA DEL SESGO:

    Sesgo = β₂ × δ₁
    
    donde δ₁ es el coeficiente de una regresión de faminc sobre cigs

En este caso:
• β₂ (efecto de faminc) es POSITIVO ({beta2_mult:.3f})
• Corr(cigs, faminc) es NEGATIVA ({correlacion:.3f})
• Por tanto, el sesgo es NEGATIVO

Esto significa que el modelo simple SOBREESTIMA el efecto negativo de cigs
(el coeficiente es MÁS negativo de lo que debería).

Sin embargo, el cambio es relativamente pequeño ({cambio_pct:.1f}%), lo que indica:
1. La correlación entre cigs y faminc no es muy fuerte, O
2. El efecto de faminc sobre bwght es modesto

✅ CONCLUSIÓN: El efecto negativo del tabaquismo es ROBUSTO.
   Existe sesgo, pero no es dramático en este caso.
""")

In [ ]:
# ============================================================================
# VISUALIZACIÓN - EJERCICIO 2
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Comparación de líneas de regresión
ax1 = axes[0]
x_cigs = np.linspace(0, 50, 100)
y_simple = beta0_simple + beta1_simple * x_cigs
y_mult = beta0_mult + beta1_mult * x_cigs + beta2_mult * bwght['faminc'].mean()

ax1.plot(x_cigs, y_simple, label=f'Modelo Simple: β₁ = {beta1_simple:.3f}', linewidth=2)
ax1.plot(x_cigs, y_mult, label=f'Modelo Múltiple: β₁ = {beta1_mult:.3f}', linewidth=2, linestyle='--')
ax1.set_xlabel('Cigarrillos por día')
ax1.set_ylabel('Peso al nacer predicho (onzas)')
ax1.set_title('Comparación: Efecto del Tabaquismo\n(el modelo múltiple usa faminc = media)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Dispersión
ax2 = axes[1]
ax2.scatter(bwght['cigs'], bwght['bwght'], alpha=0.3, s=20)
ax2.plot(x_cigs, y_simple, color='red', linewidth=2, label='Línea de regresión (simple)')
ax2.set_xlabel('Cigarrillos por día')
ax2.set_ylabel('Peso al nacer (onzas)')
ax2.set_title('Peso al Nacer vs Tabaquismo', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 📘 EJERCICIO 3: Inferencia Estadística
## Riqueza Financiera, Ingreso y Edad (Capítulo 4)

**Contexto:** Análisis de los determinantes de la riqueza financiera en hogares estadounidenses.

**Modelo:** `nettfa = β₀ + β₁·inc + β₂·age + u`

**Variables:**
- `nettfa`: Riqueza financiera neta (miles de $)
- `inc`: Ingreso familiar anual (miles de $)
- `age`: Edad
- `fsize`: Tamaño de la familia

⚠️ **NOTA:** Usaremos solo hogares de una persona (`fsize = 1`)

In [ ]:
# ============================================================================
# EXPLORACIÓN DE DATOS
# ============================================================================

print("=" * 60)
print("EJERCICIO 3: INFERENCIA - Riqueza, Ingreso y Edad")
print("=" * 60)

print(f"\nDataset completo: {ksubs.shape[0]} observaciones")
print("\nPrimeras observaciones:")
display(ksubs[['nettfa', 'inc', 'age', 'fsize', 'e401k']].head())

### Pregunta a) Número de hogares unipersonales

In [ ]:
# ============================================================================
# PREGUNTA a) FILTRAR HOGARES UNIPERSONALES
# ============================================================================

# Filtramos para quedarnos SOLO con hogares de una persona
# En pandas, usamos == para comparar (no =)
single = ksubs[ksubs['fsize'] == 1].copy()

print(f"""
📌 HOGARES UNIPERSONALES
══════════════════════════════════════════════════════════════

Hogares de una sola persona (fsize = 1): {len(single)}

Esto representa el {len(single)/len(ksubs)*100:.1f}% del total.

¿Por qué filtrar por hogares unipersonales?
• Elimina heterogeneidad por tamaño familiar
• Simplifica la interpretación (no hay que dividir entre miembros)
• Reduce posibles problemas de endogeneidad
""")

print("Estadísticas descriptivas (hogares unipersonales):")
display(single[['nettfa', 'inc', 'age']].describe())

### Pregunta b) Estimación del modelo

In [ ]:
# ============================================================================
# PREGUNTA b) ESTIMACIÓN DEL MODELO
# ============================================================================

# Modelo: nettfa = β₀ + β₁·inc + β₂·age + u
y = single['nettfa']
X = sm.add_constant(single[['inc', 'age']])

modelo3 = sm.OLS(y, X).fit()

print("\n" + "=" * 60)
print("RESULTADOS DE LA REGRESIÓN")
print("=" * 60)
print(modelo3.summary())

In [ ]:
# Extraemos los parámetros
beta0 = modelo3.params['const']
beta1 = modelo3.params['inc']
beta2 = modelo3.params['age']
se0 = modelo3.bse['const']
se1 = modelo3.bse['inc']
se2 = modelo3.bse['age']
r2 = modelo3.rsquared

print(f"""
📌 ECUACIÓN ESTIMADA
══════════════════════════════════════════════════════════════

nettfa = {beta0:.2f} + {beta1:.3f}·inc + {beta2:.3f}·age
         ({se0:.2f})  ({se1:.3f})    ({se2:.3f})

n = {int(modelo3.nobs)}, R² = {r2:.3f}

══════════════════════════════════════════════════════════════

📐 INTERPRETACIÓN DE LOS COEFICIENTES DE PENDIENTE:

β₁ (inc) = {beta1:.3f}:
   Por cada $1,000 adicionales de ingreso familiar anual,
   la riqueza financiera neta aumenta en ${beta1*1000:.0f},
   manteniendo la edad CONSTANTE (ceteris paribus).

β₂ (age) = {beta2:.3f}:
   Por cada año adicional de edad,
   la riqueza financiera neta aumenta en ${beta2*1000:.0f},
   manteniendo el ingreso CONSTANTE.

✅ Ambos efectos son POSITIVOS y tienen sentido económico:
   • Mayor ingreso → mayor capacidad de ahorro
   • Mayor edad → más tiempo para acumular riqueza
""")

### Pregunta c) Interpretación del intercepto

In [ ]:
# ============================================================================
# PREGUNTA c) INTERPRETACIÓN DEL INTERCEPTO
# ============================================================================

print(f"""
📌 INTERPRETACIÓN DEL INTERCEPTO
══════════════════════════════════════════════════════════════

El intercepto β₀ = {beta0:.2f} representa la riqueza financiera neta 
predicha para una persona con:
• inc = 0 (ingreso cero)
• age = 0 (edad cero)

❌ ¿Tiene sentido práctico? NO, por varias razones:

1. Nadie tiene edad = 0 en esta muestra (edad mínima = {single['age'].min()})
2. La combinación (inc=0, age=0) está MUY lejos de los datos observados
3. Un intercepto negativo sugiere deuda, pero no para un bebé sin ingresos

💡 IMPORTANTE: En muchos modelos, el intercepto NO tiene interpretación 
   práctica. Su función principal es permitir que la línea de regresión 
   se ajuste mejor a los datos.

   Lo relevante son los COEFICIENTES DE PENDIENTE.
""")

### Pregunta d) Prueba de hipótesis H₀: β₂ = 1

In [ ]:
# ============================================================================
# PREGUNTA d) PRUEBA DE HIPÓTESIS H₀: β₂ = 1
# ============================================================================

# Hipótesis:
# H₀: β₂ = 1 (cada año de edad aumenta la riqueza en $1,000)
# H₁: β₂ ≠ 1 (alternativa de dos colas)

# Estadístico t para hipótesis NO estándar:
# t = (β̂₂ - c) / se(β̂₂)  donde c es el valor hipotético

c = 1  # Valor bajo H₀
t_stat = (beta2 - c) / se2
df = modelo3.df_resid  # Grados de libertad: n - k - 1

# Valor-p (dos colas)
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df))

print(f"""
📌 PRUEBA DE HIPÓTESIS
══════════════════════════════════════════════════════════════

H₀: β₂ = 1  (el efecto de la edad es exactamente $1,000 por año)
H₁: β₂ ≠ 1  (alternativa de DOS colas)

══════════════════════════════════════════════════════════════

📐 CÁLCULO DEL ESTADÍSTICO t:

         β̂₂ - c       {beta2:.3f} - {c}
    t = ───────── = ─────────────── = {t_stat:.3f}
         se(β̂₂)         {se2:.3f}

Grados de libertad: df = n - k - 1 = {int(modelo3.nobs)} - 2 - 1 = {int(df)}

Valor-p (dos colas): {p_value:.4f}

══════════════════════════════════════════════════════════════

📊 DECISIÓN:

• Al nivel α = 0.01 (1%): valor-p = {p_value:.4f} > 0.01
  → NO RECHAZAMOS H₀ ✓
  
• Al nivel α = 0.05 (5%): valor-p = {p_value:.4f} > 0.05
  → NO RECHAZAMOS H₀ ✓

══════════════════════════════════════════════════════════════

✅ CONCLUSIÓN:

No hay evidencia estadística suficiente para concluir que el efecto 
de la edad sea diferente de $1,000 por año.

⚠️ NOTA: "No rechazar H₀" NO significa que β₂ = 1 sea verdadero.
   Solo significa que los datos no proveen evidencia suficiente 
   para descartarlo.
""")

### Pregunta e) Intervalo de confianza para β₁

In [ ]:
# ============================================================================
# PREGUNTA e) INTERVALO DE CONFIANZA AL 95% PARA β₁
# ============================================================================

# IC al (1-α)% = β̂ ± t(α/2, df) × se(β̂)
alpha = 0.05
t_critico = stats.t.ppf(1 - alpha/2, df)

ic_lower = beta1 - t_critico * se1
ic_upper = beta1 + t_critico * se1

# También podemos obtenerlo directamente de statsmodels
ic_statsmodels = modelo3.conf_int(alpha=0.05).loc['inc']

print(f"""
📌 INTERVALO DE CONFIANZA AL 95% PARA β₁ (coeficiente de inc)
══════════════════════════════════════════════════════════════

📐 FÓRMULA: IC = β̂₁ ± t(α/2, df) × se(β̂₁)

Cálculo:
• β̂₁ = {beta1:.4f}
• se(β̂₁) = {se1:.4f}
• t(0.025, {int(df)}) = {t_critico:.3f}

IC₉₅% = {beta1:.4f} ± {t_critico:.3f} × {se1:.4f}
IC₉₅% = {beta1:.4f} ± {t_critico * se1:.4f}

══════════════════════════════════════════════════════════════

📊 IC₉₅% = [{ic_lower:.4f}, {ic_upper:.4f}]

Verificación con statsmodels: [{ic_statsmodels[0]:.4f}, {ic_statsmodels[1]:.4f}]

══════════════════════════════════════════════════════════════

📐 INTERPRETACIÓN:

Con 95% de confianza, el verdadero efecto de un incremento de $1,000 
en el ingreso familiar sobre la riqueza financiera neta está entre 
${ic_lower*1000:.0f} y ${ic_upper*1000:.0f}.

💡 NOTAS IMPORTANTES:

1. El intervalo NO incluye el cero 
   → β₁ es estadísticamente SIGNIFICATIVO al 5%
   
2. El intervalo es relativamente estrecho 
   → estimación PRECISA
   
3. "95% de confianza" significa: si repitiéramos el muestreo muchas 
   veces, el 95% de los intervalos calculados contendrían el 
   verdadero parámetro.
""")

In [ ]:
# ============================================================================
# VISUALIZACIÓN - EJERCICIO 3
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Gráfico 1: nettfa vs inc
ax1 = axes[0, 0]
ax1.scatter(single['inc'], single['nettfa'], alpha=0.3, s=20)
ax1.set_xlabel('Ingreso (miles de $)')
ax1.set_ylabel('Riqueza financiera neta (miles de $)')
ax1.set_title('Riqueza vs Ingreso', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Gráfico 2: nettfa vs age
ax2 = axes[0, 1]
ax2.scatter(single['age'], single['nettfa'], alpha=0.3, s=20)
ax2.set_xlabel('Edad')
ax2.set_ylabel('Riqueza financiera neta (miles de $)')
ax2.set_title('Riqueza vs Edad', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Gráfico 3: Distribución de residuales
ax3 = axes[1, 0]
residuales = modelo3.resid
ax3.hist(residuales, bins=50, edgecolor='black', alpha=0.7)
ax3.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Residuales')
ax3.set_ylabel('Frecuencia')
ax3.set_title('Distribución de Residuales', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Gráfico 4: Intervalos de confianza
ax4 = axes[1, 1]
coefs = ['inc', 'age']
estimates = [beta1, beta2]
ci_lower_list = [modelo3.conf_int().loc['inc'][0], modelo3.conf_int().loc['age'][0]]
ci_upper_list = [modelo3.conf_int().loc['inc'][1], modelo3.conf_int().loc['age'][1]]
errors = [[est - low for est, low in zip(estimates, ci_lower_list)],
          [up - est for est, up in zip(estimates, ci_upper_list)]]

ax4.errorbar(coefs, estimates, yerr=errors, fmt='o', capsize=5, capthick=2, markersize=10)
ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax4.axhline(y=1, color='red', linestyle=':', alpha=0.7, label='H₀: β = 1')
ax4.set_ylabel('Coeficiente estimado')
ax4.set_title('Coeficientes con IC 95%', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 📚 Resumen de Conceptos Clave

## Ejercicio 1 - Regresión Simple
- ✅ Interpretación de coeficientes (β₀ y β₁)
- ✅ El intercepto tiene sentido cuando X=0 está en rango razonable
- ✅ R² mide el poder explicativo, pero R² bajo ≠ modelo inútil
- ✅ **Siempre usar `sm.add_constant()` para incluir el intercepto**

## Ejercicio 2 - Regresión Múltiple
- ✅ **Sesgo por variable omitida**: Sesgo = β₂ × Corr(X₁, X₂)
- ✅ Coeficientes tienen interpretación **CETERIS PARIBUS**
- ✅ Comparar modelos ayuda a entender la robustez de los resultados
- ✅ Agregar variables de control puede cambiar los coeficientes

## Ejercicio 3 - Inferencia
- ✅ Prueba t para H₀: β = c:  `t = (β̂ - c) / se(β̂)`
- ✅ Valor-p: probabilidad de observar algo tan extremo bajo H₀
- ✅ IC al 95%: `β̂ ± t(0.025, df) × se(β̂)`
- ✅ **"No rechazar H₀" ≠ "H₀ es verdadera"**
- ✅ Significancia estadística ≠ significancia práctica

---

## 📦 Librerías Útiles

| Librería | Uso |
|----------|-----|
| `pandas` | Manipulación de datos |
| `statsmodels` | Regresiones y pruebas de hipótesis |
| `scipy.stats` | Distribuciones y valores críticos |
| `matplotlib/seaborn` | Visualización |
| `wooldridge` | Datasets del libro |